In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout

# Read the data from a CSV file
df = pd.read_csv('stockprices.csv')

# Convert the 'Date' column to a datetime object
df['Date'] = pd.to_datetime(df['Date'])

# Sort the data by date in ascending order
df = df.sort_values('Date')

# Set the 'Date' column as the index
df.set_index('Date', inplace=True)

# Drop any unnecessary columns
df = df[['Open', 'High', 'Low', 'Close*', 'Adj Close**', 'Volume']]

# Remove commas and convert values to float
df = df.replace(',', '', regex=True).astype(float)

# تقسیم دیتا به داده‌های آموزش و آزمون

train_size = int(len(df) * 0.8)
train_data = df.iloc[:train_size, :]
test_data = df.iloc[train_size:, :]

# نرمال‌سازی داده‌ها
scaler = MinMaxScaler(feature_range=(0, 1))
train_data = scaler.fit_transform(train_data)
test_data = scaler.transform(test_data)



# محدود کردن داده‌های آموزش به یک چهارم
#step = 4  # محدود کننده تعداد داده‌های ورودی
X_train = []
Y_train = []
for i in range(60, len(train_data)):
    X_train.append(train_data[i - 60:i, :])
    Y_train.append(train_data[i, 0])
X_train, Y_train = np.array(X_train), np.array(Y_train)

X_test = []
Y_test = []
for i in range(60, len(test_data)):
    X_test.append(test_data[i - 60:i, :])
    Y_test.append(test_data[i, 0])
X_test, Y_test = np.array(X_test), np.array(Y_test)

# ساخت مدل شبکه عصبی با دو لایه Dense و یک لایه Dropout
model = Sequential()
model.add(Dense(units=64, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(units=32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=1))

# کامپایل مدل شبکه عصبی

model.compile(loss='mean_squared_error', optimizer='adam')


# آموزش مدل شبکه عصبی
model.fit(X_train, Y_train, epochs=100, batch_size=32, verbose=1)

# ارزیابی مدل
train_loss = model.evaluate(X_train, Y_train, verbose=0)
test_loss = model.evaluate(X_test, Y_test, verbose=0)

print("Train Loss:", train_loss)
print("Test Loss:", test_loss)



In [ ]:
from keras.models import Sequential
from keras.layers import Dense,LSTM, Dropout
# تقسیم دیتا به داده‌های آموزش و آزمون
train_size = int(len(df) * 0.8)
train_data = df.iloc[:train_size, :]
test_data = df.iloc[train_size:, :]

# نرمال‌سازی داده‌ها
scaler = MinMaxScaler(feature_range=(0, 1))
train_data_scaled = scaler.fit_transform(train_data.iloc[:, 0].values.reshape(-1, 1))
test_data_scaled = scaler.transform(test_data.iloc[:, 0].values.reshape(-1, 1))

# محدود کردن داده‌های آموزش به یک چهارم
X_train = []
Y_train = []
for i in range(60, len(train_data_scaled)):
    X_train.append(train_data_scaled[i - 60:i, 0])
    Y_train.append(train_data_scaled[i, 0])
X_train, Y_train = np.array(X_train), np.array(Y_train)

X_test = []
Y_test = []
for i in range(60, len(test_data_scaled)):
    X_test.append(test_data_scaled[i - 60:i, 0])
    Y_test.append(test_data_scaled[i, 0])
X_test, Y_test = np.array(X_test), np.array(Y_test)

# بازسازی شکل ورودی
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_train.shape[1], 1))

# ساخت مدل شبکه عصبی با دو لایه LSTM و یک لایه Dropout
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(units=1))

# کامپایل مدل شبکه عصبی
model.compile(loss='mean_squared_error', optimizer='adam',run_eagerly=True)

# آموزش مدل شبکه عصبی
model.fit(X_train, Y_train, epochs=100, batch_size=32, verbose=1)
# ارزیابی مدل
train_loss = model.evaluate(X_train, Y_train, verbose=0)
test_loss = model.evaluate(X_test, Y_test, verbose=0)

print("Train Loss:", train_loss)
print("Test Loss:", test_loss)




In [ ]:
print(df)

                Open      High       Low    Close*  Adj Close**       Volume
Date                                                                        
2022-05-31  33160.59  33240.22  32752.34  32990.12     32990.12  533560000.0
2022-06-01  33156.31  33272.34  32584.76  32813.23     32813.23  338210000.0
2022-06-02  32809.01  33248.61  32509.43  33248.28     33248.28  333210000.0
2022-06-03  32986.32  33135.61  32839.21  32899.70     32899.70  298570000.0
2022-06-06  33032.04  33235.37  32819.50  32915.78     32915.78  253010000.0
...              ...       ...       ...       ...          ...          ...
2023-05-22  33408.54  33512.30  33207.85  33286.58     33286.58  268900000.0
2023-05-23  33190.60  33310.17  33013.29  33055.51     33055.51  291550000.0
2023-05-24  33021.76  33031.75  32752.44  32799.92     32799.92  246180000.0
2023-05-25  32854.26  32870.43  32586.56  32764.65     32764.65  364670000.0
2023-05-26  32795.50  33162.06  32795.50  33093.34     33093.34  330290000.0

In [ ]:
df.head()

,Open,High,Low,Close*,Adj Close**,Volume
Date,,,,,,
2022-05-31,33160.59,33240.22,32752.34,32990.12,32990.12,533560000.0
2022-06-01,33156.31,33272.34,32584.76,32813.23,32813.23,338210000.0
2022-06-02,32809.01,33248.61,32509.43,33248.28,33248.28,333210000.0
2022-06-03,32986.32,33135.61,32839.21,32899.70,32899.70,298570000.0
2022-06-06,33032.04,33235.37,32819.50,32915.78,32915.78,253010000.0
